# MRI Segmentation #

## Motivation ##
In 2017, dementia became the biggest killer in the UK, overcoming vastly feared diseases such as heart disease and cancer, being responsible for a devastating 70,366 deaths in Britain alone. Dementia is a group of diseases characterised by cognitive impairment, which has two common symptoms; memory loss and clouded judgement, this can advance to an extent that makes it difficult for patients to execute normal daily tasks such as going to work or even preparing a meal. Alzheimer’s disease (AD) is the most common type of senile dementia in the UK. AD is a neurodegenerative disease that was named after the German clinical psychiatrist and neuroanatomist, Alois Alzheimer, who on November 3rd, 1906 reported “A peculiar severe disease process of the cerebral cortex”, being the first person to describe the disease’s symptoms and common hallmarks, the infamous beta amyloid plaques (Aβ plaques) and neurofibrillary tangles. These plaques and tangles described by Alois Alzheimer impair the functioning of previously healthy neurons which lose synaptic connections to one another and eventually die. Initially, the part of the brain that suffers the most from these changes is the hippocampus, which is essential for the formation of new memories. The death of neurons then spreads across the brain, resulting in atrophy of the white matter, as the gyri decrease in size, and the vacuoles and sulci to become larger, thus the cross-section of the brain becomes significantly smaller as the disease progresses. All these changes in the structure of the brain lead to the symptoms commonly related to AD such as forgetting a family member’s name or finding it hard to locate themselves. This atrophy of the white matter results in a grey matter to white matter ratio that is distinguishable to that of a 'healthy' brain. The purpose of this project is to provide an automated way of diagnosing AD through this grey matter to white matter ratio.

In [49]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 安装第三方脑区提取工具
pip install git+https://github.com/rockstreamguy/deepbrain.git#egg=deepbrain

## Helper for Otsu Algorithm ##

In [51]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import cv2
import pylab as plt
import numpy as np

def varianceCalculate(average, histgram):
    #计算方差
    variance = 0
    for i in range(len(histgram)):
        variance += (histgram[i] - average) ** 2

    variance /= len(histgram)

    return variance

def averageAndpixelSumCalculate(histgram):
    #计算全部像素的平均值
    average = pixelSum = 0
    for i in range(len(histgram)):
        pixelSum += histgram[i]         #像素总数
        brightnessValue = histgram[i] * i   #辉度计算

    average = brightnessValue / len(histgram)

    return pixelSum, average

def within_betweenCV(pixelSum1, average1, variance1, pixelSum2, average2, variance2):
    #返回类间分布和类内分布

    betweenClassVariance = (pixelSum1 * pixelSum2 * ((average1 - average2) ** 2) ) / ((pixelSum1 + pixelSum2) ** 2) #类间分布
    withinClassVariance = (pixelSum1 * variance1 + pixelSum2 * variance2) / (pixelSum1 + pixelSum2) #类内分布

    return betweenClassVariance, withinClassVariance

def calculateAll(blackList, whiteList):
    #执行计算过程
    b_size, b_average = averageAndpixelSumCalculate(blackList)
    w_size, w_average = averageAndpixelSumCalculate(whiteList)

    b_variance = varianceCalculate(b_average, blackList)
    w_variance = varianceCalculate(w_average, whiteList)

    betweenCV, withinCV = within_betweenCV(b_size, b_average, b_variance, w_size, w_average, w_variance)

    totalVariance = betweenCV + withinCV #总方差
    separationMetrics = betweenCV / (totalVariance - betweenCV) #类间方差与类内方差的比率

    return separationMetrics


## Functions for brain extraction and segmentation ##

In [111]:
def brain_extraction(volume):
  # 调用第三方算法对脑区进行提取
  # 详细见https://github.com/rockstreamguy/deepbrain

  ext = Extractor()
  prob = ext.run(volume) 
  mask = prob > 0.5
  brain_volume = np.multiply(mask, volume)

  return brain_volume

def find_threshold(img):
  # 找到最佳分割阈值

  #计算直方图
  hist, bin_edges = np.histogram(img, bins=256, range=[0, 256], density=True)
  
  #计算分离度
  size = 256
  listSM = [0 for i in range(size)]                       
  for i in range(size):
    if i != 0 and i != size-1:
      blackList = hist[0: i]                    #亮度较低的部分
      whiteList = hist[i: size]                   #亮度较高的部分
      listSM[i] = calculateAll(blackList, whiteList)         #分散度
    elif i == 0 or i == size-1:
      listSM[i] = 0                           #无法将其分为两个列表时的异常处理

  #从最大分离度中找到阈值
  maxValue = 0
  maxValueIndex = 0

  #保存最大分散值的索引
  for i in range(size):   
    if listSM[i] > maxValue:
      maxValue = listSM[i]
      maxValueIndex = i     

  return hist, maxValueIndex

def binarization_otsu(img, maxValueIndex):
  #根据获得的阈值对图像进行二值化
  output_otsu = np.zeros((len(img), len(img[0])))    #使用数组输出大津法的结果
  for i in range(len(img)):
    for j in range(len(img[0])):
      if img[i][j] > maxValueIndex:
        output_otsu[i][j] = 255      #大于阈值的部分设为黑色
      else:
        output_otsu[i][j] = 0       #小于阈值的部分设为白色
  return output_otsu

def binarization_avg(img, hist):
  #以直方图中亮度值的中间值作为阈值创建二值化图像
  average_histgram = int(len(hist) / 2)           #获取直方图长度的一半
  output_average = img.copy()                #用于简单二值化输出的阵列
  output_average[output_average >= average_histgram] = 255  
  output_average[output_average < average_histgram] = 0        #小于阈值的部分设为白色
  
  return output_average

from skimage.morphology import erosion, opening
from skimage.morphology import disk

def result_improvement(origin):
  #基于形态学操作对图形进行微调
  #操作列表见 https://scikit-image.org/docs/dev/auto_examples/applications/plot_morphology.html

  #形态学操作基本单元设置
  selem = disk(1)

  #得到腐蚀操作结果
  erosed = erosion(origin, selem)

  #二值化
  for i in range(len(erosed)):
    for j in range(len(erosed[0])):
      if erosed[i][j] > 0:
        erosed[i][j] = 255      #大于阈值的部分设为黑色
      else:
        erosed[i][j] = 0       #小于阈值的部分设为白色

  return erosed

def show_comparison(origin, brain, otsu):
  #显示大脑提取，大脑分割结果
  fig, ax = plt.subplots(2, 2, figsize=(10,10))

  #隐藏网格线
  plt.ioff()
  plt.grid(False)
  plt.grid(b=None)

  ax[0, 0].imshow(origin, cmap='gray')        #原图
  ax[0, 0].grid(False)

  brain[brain > 0] = 255
  ax[0, 1].imshow(brain, cmap='gray')         #大脑提取
  ax[0, 1].grid(False)  

  ax[1, 0].imshow(otsu, cmap='gray')         #大津法阈值
  ax[1, 0].grid(False)

  ax[1, 1].imshow(origin, cmap='gray')        #堆叠图
  ax[1, 1].imshow(otsu, 'jet', alpha=0.5)      
  ax[1, 1].grid(False)
  plt.savefig('hehe.png')
  plt.close(fig)


def calcWGRatio(whole_img, white_img):

  # 计算图像中白点和黑点的数量
  greyCount = 0
  whiteCount = 0

  for i in range(len(whole_img)):
    for j in range(len(whole_img[0])):
      if whole_img[i][j] == 255:
        if whole_img[i][j] == white_img[i][j]:
          whiteCount += 1
        else:
          greyCount += 1
  
  return whiteCount, greyCount


## Main process flow ##

In [112]:
import numpy as np
import nibabel as nib
import matplotlib.pyplot as plt
import seaborn as sns
from deepbrain import Extractor
from google.colab import output

sns.set_style('darkgrid')

img_1_1 = nib.load('/content/drive/MyDrive/Road Recommendation/subject1_01.nii')
img_data_1_1 = img_1_1.get_fdata()

brain_volume = brain_extraction(img_data_1_1)
slice_num = list(brain_volume.shape)[2]
total_white_num = 0
total_grey_num = 0

for i in range(slice_num):

  output.clear()
  print('Now computing ' + str(i) + 'th of total ' + str(slice_num) + ' brain slices')

  b_slice = brain_volume[:, :, i]

  # 使用大津算法得到最佳阈值
  hist, threshold_index = find_threshold(b_slice)

  # 使用大津算法得到的最佳阈值进行图像二值化
  binary_otsu = binarization_otsu(b_slice, threshold_index)

  # 得到形态学微调结果 (可选)
  improved_img = result_improvement(binary_otsu)

  # 存储或者显示图像
  show_comparison(img_data_1_1[:, :, i], b_slice, improved_img)

  # 分别计算白点和黑点的数量
  whiteNum, greyNum = calcWGRatio(b_slice, improved_img)
  
  total_white_num += whiteNum
  total_grey_num += greyNum

# 输出最终结果
print('Ratio of grey matter to white matter ' + str(total_grey_num / total_white_num))


Now computing 123th of total 124 brain slices
Ratio of grey matter to white matter 2.0934071628827726


## For visualize the otsu's result in histogram ##

In [113]:
def plot_hist(hist, maxValueIndex):

  #输出直方图
  plt.plot(hist)                                      
  plt.axvline(x=maxValueIndex, color='red', label='otsu')          #大津法阈值显示
  plt.axvline(x=average_histgram, color='green', label='average')      #静态阈值显示
  plt.legend(loc='upper right')                       
  plt.title("histgram of brightness")                    
  plt.xlabel("brightness")                             
  plt.ylabel("frequency")                               
  plt.xlim([0, 256])                     
  plt.show()


## The separation of white and grey matter ##

In order to analyse the ratio of white and grey matter inside each  patients brain the respective regions must be separated.

Over the years there have been many techniques developed to allow people to convert normal greyscale images into binary images (black and white). The complexity and accuracy of these techniques vary rather a lot.

In this project we hope to use Otsu's method of image segmentation to binarise the MRI scans of patients one and two, thus allowing the simple comparison of white to grey matter ratios to take place.



Otsu's method is essentially just an algorithm that finds the optimal threshold intensity (the optimal point at which to start calling the background the foreground).

To do this it calculates something called the between class variance (${σ_b}^2$):

$ {σ_b}^2 = W_b*W_f*(μ_b - μ_f)^2 $


subscripts b and f denote background and foreground respectively.

W stands for the weight which is simply the number of pixels in the given section divided by the total number of pixels.


μ stands for the mean intensity, given by the following equation:


$ μ_b = Σ (GL*Intensity) / n.pix_b $

where GL stands for the grey level (a completely black pixel has a GL value of 0, lighter pixels having larger and larger GL values)


Putting all of these components into the between class variance equation a value is obtained, the algorithm then repeats this for every possible threshold level. The level that gives the largest between class variance is taken as the best possible threshold level for turning a grey scale image into a binary one.

